<a href="https://colab.research.google.com/github/JasCapasci/PapyrusNebulae_2023/blob/main/PapyrusNebulae_2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


After spending a good chunk of time Googling and reading Stack Overflow I got to know that the pdfservices-sdk package is not available directly via pip for installation in Google Colab. But I still tried to utilize the Adobe PDF Services Extract API in Google Colab by making HTTP requests to the API endpoints directly.

In [8]:
!pip install requests

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [9]:
import os
import csv

In [10]:
import requests
import json

In [11]:
import logging

In [12]:
client_id = '1e57004a48674d1fb5af2c42e971bcad'
client_secret = 'p8e-MYlNZlKZHm_4iOE0e_Y0Rss9Lhr_Awff'
organization_id = '9AA21EBD648726540A495F85@AdobeOrg'
account_id = '92561EBE648728580A495EA7@techacct.adobe.com'
private_key_file = '/content/drive/MyDrive/adobe-dc-pdf-services-sdk-python-samples/private.key'

In [13]:
extract_api_url = 'https://pdfservices.adobe.io'

In [14]:
def get_access_token():
    token_url = f'{extract_api_url}/oauth/refresh'
    payload = {
        'client_id': client_id,
        'client_secret': client_secret,
        'jwt_token': ''
    }
    response = requests.post(token_url, json=payload)
    response_data = response.json()
    access_token = response_data['access_token']
    return access_token

In [15]:
folder_path = '/content/drive/MyDrive/InvoicesData/TestDataSet'

In [18]:
input_file_paths = []
for file_name in os.listdir(folder_path):
    if file_name.endswith('.pdf'):
        file_path = os.path.join(folder_path, file_name)
        input_file_paths.append(file_path)

In [19]:
def extract_invoice_data(input_file_paths):
    extract_url = f'{extract_api_url}/pdfExtract/operations'
    access_token = get_access_token()

In [20]:
for file_path in input_file_paths:
    # Set the operation options for each file
    operation_options = {
        'input': {
            'file': {
                'filename': file_path
            }
        },
        'output': {
            'format': 'csv'
        },
        'elements': [
            {
                'type': 'text'
            },
            {
                'type': 'table'
            }
        ]
      }

In [ ]:
# Make the API request
    response = requests.post(extract_url, headers=headers, json=operation_options)

    # Check if the request was successful
    if response.status_code == 202:
        operation_id = response.json()['operationId']
        return operation_id
    else:
        error_message = response.json()['error']['message']
        raise Exception(f'Error extracting data: {error_message}')

# Function to check the extraction status
def check_extraction_status(operation_id):
    status_url = f'{extract_api_url}/pdfExtract/operations/{operation_id}'
    access_token = get_access_token()

    # Set the headers with the access token
    headers = {
        'Content-Type': 'application/json',
        'Authorization': f'Bearer {access_token}'
    }

    # Make the API request
    response = requests.get(status_url, headers=headers)

    # Check if the request was successful
    if response.status_code == 200:
        status = response.json()['status']
        return status
    else:
        error_message = response.json()['error']['message']
        raise Exception(f'Error checking extraction status: {error_message}')

# Function to download the extracted data
def download_extracted_data(operation_id, output_file_path):
    download_url = f'{extract_api_url}/pdfExtract/operations/{operation_id}/output'
    access_token = get_access_token()

    # Set the headers with the access token
    headers = {
        'Content-Type': 'application/json',
        'Authorization': f'Bearer {access_token}'
    }

    # Make the API request
    response = requests.get(download_url, headers=headers)

    # Check if the request was successful
    if response.status_code == 200:
        with open(output_file_path, 'wb') as output_file:
            output_file.write(response.content)
        print(f'Extracted data downloaded to {output_file_path}')
    else:
        error_message = response.json()['error']['message']
        raise Exception(f'Error downloading